# 正则化示例：Ridge、Lasso、Elastic Net

## 学习目标

通过本notebook，你将学会：
- 理解正则化的原理和作用
- 使用Ridge、Lasso、Elastic Net回归
- 理解正则化路径（系数随alpha的变化）
- 理解特征选择的过程
- 选择合适的正则化参数

## 课程概述

本notebook将深入展示正则化对模型的影响，包括Ridge、Lasso和Elastic Net三种方法。


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_regression

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

# 设置matplotlib在notebook中内联显示
%matplotlib inline

print("环境准备完成！")


## 1. 理论回顾

### 正则化的原理

**正则化**是在损失函数中添加惩罚项，防止模型过拟合。

**Ridge回归（L2正则化）**：
- 惩罚项：$\alpha \sum_{i=1}^{n} w_i^2$
- 效果：缩小系数，但不为0
- 适用：特征较多，需要保留所有特征

**Lasso回归（L1正则化）**：
- 惩罚项：$\alpha \sum_{i=1}^{n} |w_i|$
- 效果：部分系数变为0（特征选择）
- 适用：需要特征选择，去除不重要的特征

**Elastic Net**：
- 结合L1和L2正则化
- 在特征相关时更稳定


## 2. 生成数据

生成一个容易过拟合的数据集（特征数接近样本数）。


In [ ]:
# 生成数据（特征多于样本，模拟过拟合场景）
# 只有5个特征真正有用，其他15个是噪声
X, y = make_regression(
    n_samples=100,
    n_features=20,
    n_informative=5,  # 只有5个特征真正有用
    noise=10,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 标准化（对正则化很重要）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"数据信息:")
print(f"  训练集大小: {X_train.shape[0]}")
print(f"  测试集大小: {X_test.shape[0]}")
print(f"  特征数量: {X_train.shape[1]}")
print(f"  注意：特征数量接近样本数量，容易过拟合")


## 3. 对比不同正则化方法

对比Ridge、Lasso、Elastic Net三种正则化方法。


In [ ]:
models = {
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Lasso (α=0.1)': Lasso(alpha=0.1),
    'Elastic Net (α=0.1, l1_ratio=0.5)': ElasticNet(alpha=0.1, l1_ratio=0.5)
}

results = {}

print("=" * 70)
print("正则化方法对比")
print("=" * 70)
print(f"\n{'模型':<35} {'训练MSE':<15} {'测试MSE':<15} {'R²':<15} {'非零系数':<15}")
print("-" * 95)

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    n_nonzero = np.sum(np.abs(model.coef_) > 1e-5)  # 非零系数数量
    
    results[name] = {
        'model': model,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'test_r2': test_r2,
        'n_nonzero': n_nonzero,
        'coef': model.coef_
    }
    
    print(f"{name:<35} {train_mse:<15.4f} {test_mse:<15.4f} {test_r2:<15.4f} {n_nonzero:<15}")

print("\n观察：")
print("- Ridge：所有特征都保留，系数缩小但不为0")
print("- Lasso：进行特征选择，部分系数变为0")
print("- Elastic Net：结合两者优点")


## 4. 正则化路径分析

绘制正则化路径，观察系数随alpha的变化。


In [ ]:
print("=" * 70)
print("正则化路径分析")
print("=" * 70)

alphas = np.logspace(-3, 2, 50)  # 从0.001到100

# Ridge路径
ridge_coefs = []
ridge_test_mse = []
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    ridge_coefs.append(ridge.coef_)
    y_pred = ridge.predict(X_test_scaled)
    ridge_test_mse.append(mean_squared_error(y_test, y_pred))

# Lasso路径
lasso_coefs = []
lasso_test_mse = []
for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train_scaled, y_train)
    lasso_coefs.append(lasso.coef_)
    y_pred = lasso.predict(X_test_scaled)
    lasso_test_mse.append(mean_squared_error(y_test, y_pred))

ridge_coefs = np.array(ridge_coefs)
lasso_coefs = np.array(lasso_coefs)

# 可视化
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Ridge系数路径
ax = axes[0, 0]
for i in range(ridge_coefs.shape[1]):
    ax.plot(alphas, ridge_coefs[:, i], alpha=0.7, linewidth=1.5)
ax.set_xscale('log')
ax.set_xlabel('Alpha (正则化参数)', fontsize=12)
ax.set_ylabel('系数值', fontsize=12)
ax.set_title('Ridge回归 - 系数路径', fontsize=14)
ax.grid(True, alpha=0.3)

# Lasso系数路径
ax = axes[0, 1]
for i in range(lasso_coefs.shape[1]):
    ax.plot(alphas, lasso_coefs[:, i], alpha=0.7, linewidth=1.5)
ax.set_xscale('log')
ax.set_xlabel('Alpha (正则化参数)', fontsize=12)
ax.set_ylabel('系数值', fontsize=12)
ax.set_title('Lasso回归 - 系数路径（注意系数变为0）', fontsize=14)
ax.grid(True, alpha=0.3)

# 测试MSE vs Alpha
ax = axes[1, 0]
ax.plot(alphas, ridge_test_mse, label='Ridge', linewidth=2)
ax.plot(alphas, lasso_test_mse, label='Lasso', linewidth=2)
ax.set_xscale('log')
ax.set_xlabel('Alpha (正则化参数)', fontsize=12)
ax.set_ylabel('测试集MSE', fontsize=12)
ax.set_title('测试集MSE vs Alpha', fontsize=14)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

# 非零系数数量 vs Alpha (Lasso)
ax = axes[1, 1]
n_nonzero = [np.sum(np.abs(coef) > 1e-5) for coef in lasso_coefs]
ax.plot(alphas, n_nonzero, linewidth=2, color='orange')
ax.set_xscale('log')
ax.set_xlabel('Alpha (正则化参数)', fontsize=12)
ax.set_ylabel('非零系数数量', fontsize=12)
ax.set_title('Lasso - 非零系数数量 vs Alpha', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 找出最佳alpha
best_ridge_alpha = alphas[np.argmin(ridge_test_mse)]
best_lasso_alpha = alphas[np.argmin(lasso_test_mse)]

print(f"\n最佳Ridge alpha: {best_ridge_alpha:.4f}")
print(f"最佳Lasso alpha: {best_lasso_alpha:.4f}")


In [ ]:
# 使用最佳alpha
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.1)

ridge.fit(X_train_scaled, y_train)
lasso.fit(X_train_scaled, y_train)

# 可视化系数对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(len(ridge.coef_)), ridge.coef_, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_xlabel('特征索引', fontsize=12)
axes[0].set_ylabel('系数值', fontsize=12)
axes[0].set_title('Ridge回归 - 所有特征系数', fontsize=14)
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(range(len(lasso.coef_)), lasso.coef_, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('特征索引', fontsize=12)
axes[1].set_ylabel('系数值', fontsize=12)
axes[1].set_title('Lasso回归 - 特征选择（部分系数为0）', fontsize=14)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n特征选择对比：")
print(f"Ridge回归：所有{len(ridge.coef_)}个特征都保留")
print(f"Lasso回归：{np.sum(np.abs(lasso.coef_) > 1e-5)}个特征被保留，{np.sum(np.abs(lasso.coef_) <= 1e-5)}个特征被去除")


## 6. 总结

### 关键知识点

1. **Ridge回归**：
   - 所有特征都保留
   - 系数缩小但不为0
   - 适合特征较多但都需要保留的场景

2. **Lasso回归**：
   - 进行特征选择
   - 部分系数变为0
   - 适合需要特征选择的场景

3. **Elastic Net**：
   - 结合L1和L2正则化
   - 在特征相关时更稳定

4. **正则化参数alpha**：
   - 需要仔细调优
   - 太小：正则化效果弱
   - 太大：可能欠拟合

### 选择建议

- **需要特征选择**：使用Lasso
- **特征相关**：使用Elastic Net
- **保留所有特征**：使用Ridge
- **简单场景**：使用线性回归

---

**恭喜完成正则化的学习！** 🎉
